In [1]:
from IPython.display import clear_output, display
import sys
import os
import time
import numpy as np
import datetime

sys.path.append(os.path.join(os.environ['HOMEDRIVE'], os.environ['HOMEPATH'], 'documents\github\coinbase'))

from Trader import Trader

In [2]:
trader = Trader()

In [3]:
current_side = 'buy'
qty_per_trade = 0.05
trading_currency = 'ETH'

In [4]:
last_buy_quote = np.Inf
time_last_trade = datetime.datetime.now()

In [5]:
profit = 0
profit_margin = 1.001
sell_multiplier = 1
cycle_mins = 60*12
waiting_notify = False
selling_notify = False
total_spend = 0

while (1):
    try:
        trader.getBalances()
    except:
        time.sleep(30)
        trader.getBalances()
        
    if (current_side == 'sell') and (time_last_trade < (datetime.datetime.now() - datetime.timedelta(minutes = cycle_mins))):
        current_side = 'buy'
        sell_multiplier += 1
        selling_notify = False
        
    if (current_side == 'buy') and (time_last_trade < (datetime.datetime.now() - datetime.timedelta(minutes = cycle_mins))):
        last_buy_quote = np.Inf
        waiting_notify = False
        
    if current_side == 'buy':
        buy_q = float(trader.getBuyQuote(trading_currency, qty_per_trade))
        if (trader.balances['USD'] > buy_q) and (buy_q < (last_buy_quote - 0.05)):
            buy = trader.buy(trading_currency, qty_per_trade)
            time_last_trade = datetime.datetime.now()
            total_spend = float(buy['total']['amount'])
            current_side = 'sell'
            last_buy_quote = buy_q
            waiting_notify = False
            print('{:%H:%M:%S}: Bought {:.4f} {} for ${:.2f}'.format(datetime.datetime.now(), qty_per_trade, trading_currency, total_spend))

        elif (trader.balances['USD'] < buy_q):
            print('{:%H:%M:%S}: USD balance of ${:.2f} not enough to cover quote of ${:.2f}'.format(datetime.datetime.now(), trader.balances['USD'], buy_q))
            current_side = 'sell'
            last_buy_quote = buy_q
            time_last_trade = datetime.datetime.now()
            waiting_notify = False

        elif (buy_q > (last_buy_quote-0.05)):
            if not waiting_notify:
                waiting_notify = True
                output = 'Waiting for current buy price (${:.2f}) to drop below ${:.2f} before buying'.format(buy_q, last_buy_quote - 0.05)
                print('{:%H:%M:%S}: {}'.format(datetime.datetime.now(), output))
            
    elif current_side == 'sell':
        sell_q = float(trader.getSellQuote(trading_currency, qty_per_trade))
        if (sell_q >= (total_spend * profit_margin)):
            sell = trader.sell(trading_currency, sell_multiplier * qty_per_trade)
            time_last_trade = datetime.datetime.now()
            total_recieve = float(sell['total']['amount'])
            current_side = 'buy'
            sell_multiplier = 1
            profit += total_recieve - total_spend
            print('{:%H:%M:%S}: Sold {:.4f} {} for ${:.2f} for a profit of ${:.2f}'.format(datetime.datetime.now(), qty_per_trade, trading_currency, total_recieve, total_recieve - total_spend))
            print('Total profit so far is ${:.2f}'.format(profit))
            selling_notify = False

        else:
            if not selling_notify:
                print('{:%H:%M:%S}: Potential sell proceeds of ${:.2f} not greater than spend of ${:.2f}'.format(datetime.datetime.now(), sell_q, total_spend * profit_margin))
                selling_notify = True

    time.sleep(30)

Buying 0.05 of ETH from account_id 2d06830b-4a73-56a5-8dc8-d4eb8497c198
07:11:57: Bought 0.0500 ETH for $14.98
07:12:27: Potential sell proceeds of $11.86 not greater than spend of $14.99
19:12:11: USD balance of $7.07 not enough to cover quote of $15.13
19:12:42: Potential sell proceeds of $12.01 not greater than spend of $14.99
07:12:26: USD balance of $7.07 not enough to cover quote of $15.80
07:12:56: Potential sell proceeds of $12.66 not greater than spend of $14.99


ConnectionError: HTTPSConnectionPool(host='api.coinbase.com', port=443): Max retries exceeded with url: /v2/accounts (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000019CA9EF8F28>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))